In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Load your dataset (replace this with your actual data loading code)
# Assume you have two folders: 'authentic' and 'deepfake' containing images
# 'authentic' and 'deepfake' are class labels
data_dir = r"C:\Users\mohan\OneDrive\Documents\data - Copy"
img_size = (128, 128)
batch_size = 32

# Use ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split the dataset into training and validation sets
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Define a simple CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test Accuracy: {test_acc}')

# Save the model for later use
model.save('deepfake_detection_model.h5')


Found 15101 images belonging to 2 classes.
Found 3774 images belonging to 2 classes.
Epoch 1/10
472/472 [==============================] - 471s 995ms/step - loss: 0.5579 - accuracy: 0.6856 - val_loss: 0.6590 - val_accuracy: 0.5800
Epoch 2/10
472/472 [==============================] - 382s 810ms/step - loss: 0.3973 - accuracy: 0.8125 - val_loss: 0.6476 - val_accuracy: 0.6603
Epoch 3/10
472/472 [==============================] - 292s 618ms/step - loss: 0.3518 - accuracy: 0.8402 - val_loss: 0.6192 - val_accuracy: 0.6932
Epoch 4/10
472/472 [==============================] - 278s 588ms/step - loss: 0.3059 - accuracy: 0.8582 - val_loss: 0.6332 - val_accuracy: 0.7048
Epoch 5/10
472/472 [==============================] - 277s 587ms/step - loss: 0.2758 - accuracy: 0.8772 - val_loss: 0.6753 - val_accuracy: 0.7499
Epoch 6/10
472/472 [==============================] - 295s 624ms/step - loss: 0.2586 - accuracy: 0.8871 - val_loss: 0.6027 - val_accuracy: 0.7207
Epoch 7/10
472/472 [===================

In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('deepfake_detection_model.h5')

# Define the image file path you want to test
image_path = r"C:\Users\mohan\Downloads\GILLSARA.jpg"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Apply the same normalization used during training
img_array = img_array / 255.0

# Make predictions
predictions = model.predict(img_array)

threshold = 0.6  # Adjust this threshold based on your model's behavior
if predictions[0][0] > threshold:
    print("Authentic Image")
else:
    print("Deepfake Image")
print(f"Prediction Value: {predictions[0][0]}")


1/1 [==============================] - 0s 325ms/step
Authentic Image
Prediction Value: 0.9517403244972229


In [1]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('deepfake_detection_model.h5')

# Define the function for making predictions
def predict_deepfake(img_pil):
    # Convert PIL image to NumPy array
    img_array = np.array(img_pil)

    # Preprocess the image
    img_array = img_array / 255.0
    img_array = tf.image.resize(img_array, (128, 128))  # Resize to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions
    predictions = model.predict(img_array)

    threshold = 0.55  # Adjust this threshold based on your model's behavior
    prediction_value = predictions[0][0]

    # Interpret the predictions
    if prediction_value > threshold:
        result = "Authentic Image"
    else:
        result = "Deepfake Image"

    return result, prediction_value

# Create a Gradio interface
iface = gr.Interface(
    fn=predict_deepfake,
    inputs=gr.Image(type='pil', label="Upload an Image"),
    outputs=["text", "number"],
    live=True,
    capture_session=True
)

# Launch the interface
iface.launch()


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle